In [34]:
import fitz  # PyMuPDF

def extract_lines_from_pdf_pymupdf(pdf_path):
    """
    Extrae las líneas completas de texto de un archivo PDF utilizando PyMuPDF.
    
    Args:
        pdf_path (str): Ruta del archivo PDF.
        
    Returns:
        list: Lista de líneas extraídas del PDF.
    """
    lines = []
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text = page.get_text("text")  # Obtener texto como líneas completas
            page_lines = text.splitlines()
            lines.extend([line.strip() for line in page_lines if line.strip()])
    return lines

pdf_path = "./pruebaLLM.pdf"  
pdf_lines = extract_lines_from_pdf_pymupdf(pdf_path)
print(pdf_lines)

['Me llamo Martin, nací el 26 de noviembre de 2004', 'Me gusta el cine', 'Mi pelicula favorita es ‘Mulholland drive’ , dirigida por David Lynch']


In [35]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [36]:
from langchain_chroma import Chroma


vector_store = Chroma.from_texts(
    texts=pdf_lines,
    collection_name="pdf",
    embedding=embeddings,
)

In [37]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="pdf",
    embedding_function=embeddings,
)

In [59]:
retriever = vector_store.as_retriever()

In [57]:
question = "Como me llamo?"
docs = vector_store.similarity_search(question)
len(docs)

4

In [58]:
docs[0]

Document(metadata={}, page_content='llamo')

In [41]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# Local LLM
ollama_llm = "llama3.2"
model_local = ChatOllama(model=ollama_llm)

# Chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model_local
    | StrOutputParser()
)

In [55]:
chain.invoke("Quien es el director de mi pelicula favorita")

'El director de tu película favorita, según lo mencionado en la documentación, es David Lynch.'